In [5]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from gspread import SpreadsheetNotFound
import gspread_dataframe as gd
import gspread

## Connect to redshift

In [6]:
# Redshift connection
f = open('/Users/gabrielreynoso/Documents/Queries/db_klarprod_connection.txt', 'r')
postgres_str = f.read()
f.close()
cnx = create_engine(postgres_str)

## Google Credentials

In [9]:
# Read and Load Credentials
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/gabrielreynoso/Documents/GoogleCredentials/gabo_credentials.json')
gc = gspread.authorize(credentials)
migration_survey_sheet = gc.open('CCK Migration')

## Read Survey

In [14]:
# Read a worksheet into a Dataframe
survey_answers = pd.DataFrame(migration_survey_sheet.worksheet("Form Responses").get_all_records())

In [41]:
survey_answers = survey_answers.drop_duplicates(['referral'])

## Referral_Info

In [20]:
referral_code_query = '''
select
    id as user_id,
    referral_code
from is_kyc.user_kyc
where referral_code in {}
'''

In [21]:
users_ids = pd.read_sql_query(referral_code_query.format(tuple(survey_answers.referral.drop_duplicates().to_list())), cnx)

In [25]:
users_ids = users_ids.drop_duplicates(['user_id'])


## Migration Info

In [44]:
migration_query = '''
select
    *
from analytics_models.cck_migration_scores
where user_id in {}
'''

In [45]:
migration_info = pd.read_sql_query(migration_query.format(tuple(users_ids.user_id.to_list())), cnx)

In [47]:
migration_info = migration_info[['user_id', 'offer_amount', 'migration_reason']]

## Funnel_info

In [26]:
funnel_query = '''
select
    *
from klar.cck_funnel
where user_id in {}
'''

In [28]:
funnel_info = pd.read_sql_query(funnel_query.format(tuple(users_ids.user_id.to_list())), cnx)

In [32]:
funnel_info = funnel_info.drop(['email','phone_number'], axis=1)

In [70]:
survey_answers.columns = survey_answers.columns.str.replace('referral', 'referral_code')

## Add all info

In [64]:
complete_info = pd.merge(funnel_info,users_ids, on='user_id')

In [65]:
complete_info = pd.merge(complete_info,migration_info, on='user_id', how = 'left')

In [71]:
complete_info = pd.merge(complete_info,survey_answers, on='referral_code')

In [72]:
complete_info

,user_id,started_web_signup_mx,started_web_signup_utc,ended_web_signup_mx,ended_web_signup_utc,email_confirmed_mx,email_confirmed_utc,sms_confirmed_mx,sms_confirmed_utc,saca_risk_band,...,Submission Date,¿Tuviste problemas aceptando tu tarjeta de crédito?,¿Qué problemas tuviste?,¿Cuál otro?,Codificación 1,¿Por qué no has aceptado tu nueva tarjeta de crédito?,¿Cuál otro? 2,Codificación 2,¿Algo más que nos quieras decir?,Codificación 3
0,4e97144c-465c-4d44-b203-ce6c25d320ea,2019-10-29 23:06:12.987000,2019-10-30 05:06:12.987000,2019-10-29 23:06:12.987,2019-10-30 05:06:12.987,2019-10-29 23:16:07.679,2019-10-30 05:16:07.679,2019-10-29 23:34:49.747000,2019-10-30 05:34:49.747000,NaN,...,2023-01-26 18:01:20,NO,,,,No me interesa tener una tarjeta de crédito,,,"No, todo bien.",3
1,e31592c0-a3f5-4a4c-90e7-269511e30127,2019-11-01 10:27:31.806000,2019-11-01 16:27:31.806000,2019-11-01 10:27:31.806,2019-11-01 16:27:31.806,2019-11-01 10:30:42.185,2019-11-01 16:30:42.185,2019-11-01 10:52:35.406000,2019-11-01 16:52:35.406000,NaN,...,2023-01-29 23:38:11,NO,,,,No me interesa tener una tarjeta de crédito,,,"No, muchas gracias.",3
2,51806d93-a039-45ac-acfb-76bf8f975f15,2020-03-15 19:21:55.781000,2020-03-16 01:21:55.781000,2020-03-15 19:21:55.781,2020-03-16 01:21:55.781,2020-03-20 12:31:28.494,2020-03-20 18:31:28.494,2020-04-04 23:09:13.656000,2020-04-05 05:09:13.656000,NaN,...,2023-01-26 19:10:31,NO,,,,No me interesa tener una tarjeta de crédito,,,,
3,1c64a5cd-8221-4f45-ad7d-0053ada70c1d,2020-04-05 19:18:11.953000,2020-04-06 00:18:11.953000,2020-04-05 19:18:11.953,2020-04-06 00:18:11.953,2020-04-05 19:19:04.236,2020-04-06 00:19:04.236,2020-04-05 19:22:30.984000,2020-04-06 00:22:30.984000,NaN,...,2023-01-30 11:33:30,NO,,,,Ya tengo otra tarjeta de crédito,,,,
4,5746d020-e894-4d63-84c4-6bda3462724a,2020-04-24 08:43:50.550000,2020-04-24 13:43:50.550000,2020-04-24 08:43:50.550,2020-04-24 13:43:50.550,2020-04-24 08:47:23.755,2020-04-24 13:47:23.755,2020-04-24 08:53:26.749000,2020-04-24 13:53:26.749000,NaN,...,2023-02-04 10:30:30,NO,,,,No me interesa tener una tarjeta de crédito,,,Estoy muy contenta y feliz con mi tarjeta los ...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,e5afc28a-4677-40c3-94b6-f43efb7d0217,2022-08-23 00:05:47.787123,2022-08-23 05:05:47.787123,2022-08-23 14:03:46.430,2022-08-23 19:03:46.430,2022-08-24 10:27:01.482,2022-08-24 15:27:01.482,2022-08-24 10:33:17.108336,2022-08-24 15:33:17.108336,NaN,...,2023-01-29 13:26:11,NO,,,,Otro,si me interesa pero no es clara la opcion para...,1,,
324,57a723c5-078a-4630-aac6-f5f63bc22d3f,2022-08-25 12:26:12.184308,2022-08-25 17:26:12.184308,2022-08-25 12:33:27.097,2022-08-25 17:33:27.097,2022-08-25 12:33:51.899,2022-08-25 17:33:51.899,2022-08-25 12:34:59.900163,2022-08-25 17:34:59.900163,NaN,...,2023-01-31 15:30:12,SÍ,No pude verificar mi identidad,,,,,,No,3
325,6877ec68-23ae-429a-a9b1-97b682738742,2022-09-02 12:51:59.235179,2022-09-02 17:51:59.235179,2022-09-02 13:14:05.056,2022-09-02 18:14:05.056,2022-09-02 13:32:18.470,2022-09-02 18:32:18.470,2022-09-02 13:33:26.444645,2022-09-02 18:33:26.444645,NaN,...,2023-01-31 14:54:20,SÍ,Otro,no puedo pagar en algunos lugares dice que Nip...,6,,,,solo eso,3
326,6d67a9f1-0a35-4edc-8080-a8daf1248641,2022-09-16 17:21:27.938341,2022-09-16 22:21:27.938341,2022-09-16 17:55:07.382,2022-09-16 22:55:07.382,2022-09-21 14:38:52.230,2022-09-21 19:38:52.230,2022-09-28 15:22:22.489202,2022-09-28 20:22:22.489202,3.0,...,2023-01-31 15:26:00,SÍ,No me sale mi oferta,,,,,,No me dio la opsion de tarjeta de credito me l...,8


## Set in google sheets

In [73]:
# Append the dataframe again
gd.set_with_dataframe(migration_survey_sheet.worksheet("Complete_Info"), complete_info, row=1, col=1)